In [18]:
from gurobipy import Model, quicksum, GRB

In [28]:
Cust = set(range(1,3))
DC = Cust.copy() 
Route = {(i,j) for i in DC for j in Cust}
CT = {1,2}

T = 2
Period = list(range(1, T+1))

In [29]:
delivery = {(i,j):5 for i,j in Route}
dc_run = {(i,ct):100*ct for i in DC for ct in CT}

demand = {(j,t):10 for j in Cust for t in Period}

In [30]:
model = Model()

In [32]:
x, y = {}, {}

for i,j in Route:
    for t in Period:
        x[i,j,t] = model.addVar(vtype="C", name=f'x[{i},{j},{t}]')

for i in DC:
    for ct in CT:
        for t in Period:
            y[i,ct,t] = model.addVar(vtype="B", name=f'y[{i},{ct},{t}]')

model.update()

In [23]:
Cust_Demand_Cons, DC_Running_Cons = {}, {}

for j in Cust:
    for t in Period:
        Cust_Demand_Cons[j,t] = model.addConstr(
            quicksum(x[i,j,t] for i in DC)
            ==
            demand[j,t]
        )

for i,j in Route:
    for t in Period:
        DC_Running_Cons[i,j] = model.addConstr(
            x[i,j,t]
            <=
            quicksum(demand[j,t] for t in Period) * quicksum(y[i,ct,t]
        )
    
model.update()

In [24]:
model.setObjective(
    quicksum(delivery[i,j] * x[i,j,t] for i,j in Route for t in Period) +
    quicksum(dc_run[i] * y[i,t] for i in DC for t in Period)
    ,GRB.MINIMIZE
)

model.update()

In [25]:
model.optimize()

Optimize a model with 12 rows, 12 columns and 24 nonzeros
Variable types: 8 continuous, 4 integer (4 binary)
Coefficient statistics:
  Matrix range     [1e+00, 2e+01]
  Objective range  [5e+00, 1e+03]
  Bounds range     [1e+00, 1e+00]
  RHS range        [1e+01, 1e+01]
Presolve removed 12 rows and 12 columns
Presolve time: 0.00s
Presolve: All rows and columns removed

Explored 0 nodes (0 simplex iterations) in 0.01 seconds
Thread count was 1 (of 8 available processors)

Solution count 1: 2200 

Optimal solution found (tolerance 1.00e-04)
Best objective 2.200000000000e+03, best bound 2.200000000000e+03, gap 0.0000%


In [26]:
for v in model.getVars():
    if v.X > 0:
        print(v.VarName, v.X)

x[2,1,1] 10.0
x[2,1,2] 10.0
x[2,2,1] 10.0
x[2,2,2] 10.0
y[2,1] 1.0
y[2,2] 1.0
